In [15]:
# import libraries
import torch
from pathlib import Path

from neuralhydrology.nh_run import start_run, eval_run
import pickle
from neuralhydrology.evaluation.plots import percentile_plot_plotly

### Train model with negative logarithm of the likelihood

In [16]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path("1_basin.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("1_basin.yml"), gpu=-1)

2024-09-04 11:58:17,916: Logging to /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing_MDN/DE4_riefensbeek_soese/runs/single_basin_multi_forcing_mdn_0409_115817/output.log initialized.
2024-09-04 11:58:17,916: ### Folder structure created at /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing_MDN/DE4_riefensbeek_soese/runs/single_basin_multi_forcing_mdn_0409_115817
2024-09-04 11:58:17,916: ### Run configurations for single_basin_multi_forcing_mdn
2024-09-04 11:58:17,917: experiment_name: single_basin_multi_forcing_mdn
2024-09-04 11:58:17,917: train_basin_file: basin.txt
2024-09-04 11:58:17,917: validation_basin_file: basin.txt
2024-09-04 11:58:17,917: test_basin_file: basin.txt
2024-09-04 11:58:17,917: train_start_date: 1951-01-01 00:00:00
2024-09-04 11:58:17,918: train_end_date: 2015-09-30 00:00:00
2024-09-04 11:58:17,918: validation_start_date: 2015-10-01 00:00:00
2024-09-04 11:58:17,918: validation_end_date: 2018-09-30 00:00:00
2024

### Evaluate run on test set
The path to the best model is automatically saved in the variable best_model, and the corresponding best epoch is stored as a string in best_epoch.

In [17]:
run_dir = Path("runs/single_basin_multi_forcing_mdn_0409_115817")
eval_run(run_dir=run_dir, period="test")

2024-09-04 11:59:46,311: Using the model weights from runs/single_basin_multi_forcing_mdn_0409_115817/model_epoch020.pt
# Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]
2024-09-04 11:59:47,379: Stored metrics at runs/single_basin_multi_forcing_mdn_0409_115817/test/model_epoch020/test_metrics.csv
2024-09-04 11:59:47,381: Stored results at runs/single_basin_multi_forcing_mdn_0409_115817/test/model_epoch020/test_results.p


### Load and inspect model predictions
Load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [18]:
with open(run_dir / "test" / "model_epoch020" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

dict_keys(['DE4'])

In [19]:
results['DE4']['1D']['xr']['discharge_vol_sim']

<xarray.DataArray 'discharge_vol_sim' (date: 823, time_step: 1, samples: 1000)>
array([[[0.05130643, 0.08456612, 0.03337681, ..., 0.04541528,
         0.02285188, 0.03655028]],

       [[0.06925595, 0.05292839, 0.05345947, ..., 0.04774439,
         0.06049663, 0.03753519]],

       [[0.05036241, 0.07045519, 0.0565753 , ..., 0.05797172,
         0.06572318, 0.10364377]],

       ...,

       [[0.45875376, 0.6673503 , 0.2265377 , ..., 0.42006624,
         0.41750646, 0.6308731 ]],

       [[0.36690322, 0.30214307, 0.34834605, ..., 0.39390397,
         0.37036982, 0.33153948]],

       [[0.33319142, 0.3125166 , 0.40190566, ..., 0.3318794 ,
         0.3316791 , 0.31585494]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2018-10-01 2018-10-02 ... 2020-12-31
  * time_step  (time_step) int64 0
Dimensions without coordinates: samples

In [20]:
# Extract observations and simulations
qobs = results['DE4']['1D']['xr']['discharge_vol_obs']
qsim = results['DE4']['1D']['xr']['discharge_vol_sim']

In [21]:
fig = percentile_plot_plotly(qobs, qsim, title="Percentile Plot - Test Period")
fig.show()